## Read-in Data

In [1]:
# load libraries
import numpy as np
import pandas as pd

In [2]:
asthma = pd.read_csv("data/asthma_2020.csv")
aqi_pm25 = pd.read_csv("data/aqi_pm25_data.csv")

## Exploratory Data Analysis

In [3]:
# Check the head of each
asthma.head()

,StateFIPS,State,CountyFIPS,County,Year,Value,Data Comment,95% Confidence Interval,Confidence Interval Low,Confidence Interval High,Unnamed: 10
0,1,Alabama,1001,Autauga,2020,9.8%,NaN,(9.4% - 10.4%),9.4%,10.4%,NaN
1,1,Alabama,1003,Baldwin,2020,9.3%,NaN,(8.8% - 9.8%),8.8%,9.8%,NaN
2,1,Alabama,1005,Barbour,2020,11.1%,NaN,(10.6% - 11.6%),10.6%,11.6%,NaN
3,1,Alabama,1007,Bibb,2020,10.0%,NaN,(9.5% - 10.5%),9.5%,10.5%,NaN
4,1,Alabama,1009,Blount,2020,9.6%,NaN,(9.1% - 10.2%),9.1%,10.2%,NaN


In [4]:
aqi_pm25.head()

,Date,Source,Site ID,POC,Daily Mean PM2.5 Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,Method Code,Method Description,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,01/01/2020,AQS,60010007,3,8.6,ug/m3 LC,48,Livermore,1,100.0,...,170,Met One BAM-1020 Mass Monitor w/VSCC,41860.0,"San Francisco-Oakland-Hayward, CA",6,California,1,Alameda,37.687526,-121.784217
1,01/02/2020,AQS,60010007,3,4.5,ug/m3 LC,25,Livermore,1,100.0,...,170,Met One BAM-1020 Mass Monitor w/VSCC,41860.0,"San Francisco-Oakland-Hayward, CA",6,California,1,Alameda,37.687526,-121.784217
2,01/03/2020,AQS,60010007,3,14.2,ug/m3 LC,61,Livermore,1,100.0,...,170,Met One BAM-1020 Mass Monitor w/VSCC,41860.0,"San Francisco-Oakland-Hayward, CA",6,California,1,Alameda,37.687526,-121.784217
3,01/04/2020,AQS,60010007,3,10.9,ug/m3 LC,54,Livermore,1,100.0,...,170,Met One BAM-1020 Mass Monitor w/VSCC,41860.0,"San Francisco-Oakland-Hayward, CA",6,California,1,Alameda,37.687526,-121.784217
4,01/05/2020,AQS,60010007,3,7.8,ug/m3 LC,43,Livermore,1,100.0,...,170,Met One BAM-1020 Mass Monitor w/VSCC,41860.0,"San Francisco-Oakland-Hayward, CA",6,California,1,Alameda,37.687526,-121.784217


In [5]:
# Check for NAs
print(asthma.isnull().sum())
print(aqi_pm25.isnull().sum())

# There are quite a lot of NA values in certain columns


StateFIPS                      0
State                          0
CountyFIPS                     0
County                         0
Year                           0
Value                          0
Data Comment                3143
95% Confidence Interval        0
Confidence Interval Low        0
Confidence Interval High       0
Unnamed: 10                 3143
dtype: int64
Date                                 0
Source                               0
Site ID                              0
POC                                  0
Daily Mean PM2.5 Concentration       0
Units                                0
Daily AQI Value                      0
Local Site Name                    215
Daily Obs Count                      0
Percent Complete                     0
AQS Parameter Code                   0
AQS Parameter Description            0
Method Code                          0
Method Description                   0
CBSA Code                         5204
CBSA Name                         5204


In [34]:
# Check datatypes for each column in both dataframes
print(asthma.dtypes)
print(aqi_pm25.dtypes)

# year should be converted to datetime?

statefips                     int64
state                        object
countyfips                    int64
county                       object
year                          int64
value                        object
data_comment                float64
95%_confidence_interval      object
confidence_interval_low      object
confidence_interval_high     object
unnamed:_10                 float64
dtype: object
date                               object
source                             object
site_id                             int64
poc                                 int64
daily_mean_pm2.5_concentration    float64
units                              object
daily_aqi_value                     int64
local_site_name                    object
daily_obs_count                     int64
percent_complete                  float64
aqs_parameter_code                  int64
aqs_parameter_description          object
method_code                         int64
method_description                 object
cb

## Data Cleaning

In [6]:
# Clean column names
asthma.columns = asthma.columns.str.strip().str.lower().str.replace(" ", "_")
aqi_pm25.columns = aqi_pm25.columns.str.strip().str.lower().str.replace(" ", "_")

In [7]:
# Identify columns to join the table by in database-query.sql
print(asthma.columns.tolist())
print(aqi_pm25.columns.tolist())

['statefips', 'state', 'countyfips', 'county', 'year', 'value', 'data_comment', '95%_confidence_interval', 'confidence_interval_low', 'confidence_interval_high', 'unnamed:_10']
['date', 'source', 'site_id', 'poc', 'daily_mean_pm2.5_concentration', 'units', 'daily_aqi_value', 'local_site_name', 'daily_obs_count', 'percent_complete', 'aqs_parameter_code', 'aqs_parameter_description', 'method_code', 'method_description', 'cbsa_code', 'cbsa_name', 'state_fips_code', 'state', 'county_fips_code', 'county', 'site_latitude', 'site_longitude']


In [8]:
# will join by countyfips, one is called 'countyfips' and the other is 'county_fips_code', we need these to be consistent
aqi_pm25 = aqi_pm25.rename(columns={'county_fips_code': 'countyfips'}).rename(columns={'daily_mean_pm2.5_concentration': 'daily_mean_pm25_concentration'})

# further clean up specific column names
asthma = asthma.rename(columns={'95%_confidence_interval': 'confidence_interval'}).drop(columns='unnamed:_10')

## Export and Save

In [9]:
# export cleaned data as csv
asthma.to_csv("data/cleaned_asthma.csv", index=False)
aqi_pm25.to_csv("data/cleaned_aqi_pm25.csv", index=False)